In [38]:
import h5py
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import glob


In [39]:
home_dir = os.path.dirname(os.path.realpath('__file__'))
train_dir = home_dir+'/data/image_format/*/*'
vocab = ['electrons','protons','muons','pions','gamma']
image_size = 128

In [40]:
def get_label(file_path):
  # convert the path to a list of path components and one hot them
    parts = tf.string_split([file_path], '/')
    label = parts.values[-2] 
    matches = tf.stack([tf.equal(label, s) for s in vocab], axis=-1)
    onehot = tf.cast(matches, tf.float32)
    return onehot


In [41]:
def decode_img(img):
  # convert the compressed string to a uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [128, 128])

In [42]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [43]:
def initialize_variables():
    with tf.variable_scope('model',reuse=tf.AUTO_REUSE):
        #W1 is filter for first cnn layer. shape is (height,width,in_channel,out_channel)
        W1 = tf.get_variable('W1',shape=[4,4,3,8],initializer=tf.contrib.layers.xavier_initializer())
        W2 = tf.get_variable('W2',shape=[6,6,8,16],initializer= tf.contrib.layers.xavier_initializer())
        parameters = {'W1':W1, 'W2':W2}
    return parameters


def forward_prop(X,parameters):
    '''Arguments:
               X of shape (num_featues,batch_size)
               parameters-- dictionary of parameters
      Returns:
      logits-- without calculating activation function on the last layer as the cost function doesn't need it. 
    '''
    W1 = parameters['W1']
    W2 = parameters['W2']
    Z1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='VALID')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1,ksize=[1,8,8,1],strides=[1,8,8,1],padding='VALID') 
    # Z2 = tf.nn.conv2d(P1,W2, strides=[1,1,1,1],padding='VALID')
    # A2 = tf.nn.relu(Z2)
    # P2 = tf.nn.max_pool(A2,ksize=[1,8,8,1],strides=[1,8,8,1],padding='VALID') 
    P2 = tf.contrib.layers.flatten(P1)
    logits = tf.contrib.layers.fully_connected(P2,5,activation_fn=None)
    return logits


def compute_cost(Z,Y):
    logits = tf.transpose(Z)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels = labels))
    return cost

In [47]:
%%time
minibatch_size = 256 

epochs = 2
learning_rate=0.0001

def model(training_dataset,learning_rate=0.00002,epochs=10):   
    
    with tf.variable_scope('model',reuse=tf.AUTO_REUSE):
        with tf.device('/cpu:0'):
            batched_training_dataset = training_dataset.batch(minibatch_size).prefetch(1)
            iterator = tf.data.Iterator.from_structure(batched_training_dataset.output_types,batched_training_dataset.output_shapes)


            training_init_op = iterator.make_initializer(batched_training_dataset)
    #        validation_init_op = iterator.make_initializer(batched_validation_dataset)
            next_element = iterator.get_next()
            data = next_element     
            X = data[0]
            Y = data[1]
    #        assert(X.get_shape == (minibatch_size))
            # assert(Y.shape == (depth,minibatch_size))
        
        with tf.device('/gpu:0'):
            parameters = initialize_variables()
            logits =  forward_prop(X,parameters)    
            loss = compute_cost(logits,Y)
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
            predictions = tf.reshape(tf.argmax(logits,axis=0),shape=[-1,1])
            labels = tf.reshape(tf.argmax(Y,axis=0), shape=[-1,1])
            accuracy_calculator,accuracy_updater = tf.metrics.accuracy(predictions=predictions,labels=labels,name='metric')
            running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES,scope = 'metric')
            running_vars_initializer = tf.variables_initializer(var_list=running_vars)
            saver = tf.train.Saver(parameters, max_to_keep=1)
        
        with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.local_variables_initializer())

            for epoch in range(epochs):
                sess.run(running_vars_initializer)
                epoch_cost = 0     
           # num_minibatches  = int(training_size/minibatch_size)
                sess.run(training_init_op)
                while True:
                    try:
                        _, cost = sess.run([optimizer,loss])
                        sess.run(accuracy_updater)
                        epoch_cost = epoch_cost + cost
                    except tf.errors.OutOfRangeError:
                        if epoch%1 == 0:
                            accuracy = sess.run(accuracy_calculator)  
                            _, cost = sess.run([optimizer,loss])
                        sess.run(accuracy_updater)
                        epoch_cost = epoch_cost + cost
                    except tf.errors.OutOfRangeError:
                        if epoch%1 == 0:
                  
                         #   epoch_cost = epoch_cost/num_minibatches
                            print(' accuracy is {1} for epoch {0}'.format(epoch,accuracy))
                        break

            print("Final accuracy " + str(sess.run(accuracy_calculator)))

            saver.save(sess, home_dir+'/model_parameters')



CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


In [48]:
%%time
list_ds = tf.data.Dataset.list_files(train_dir)
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
training_dataset = list_ds.map(process_path, num_parallel_calls=5) 
model(training_dataset,learning_rate=learning_rate,epochs=epochs)
print('works')

OutOfRangeError: End of sequence
	 [[node model_8/IteratorGetNext (defined at <timed exec>:16) ]]

Errors may have originated from an input operation.
Input Source operations connected to node model_8/IteratorGetNext:
 model_8/IteratorV2 (defined at <timed exec>:11)

Original stack trace for 'model_8/IteratorGetNext':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/shubham/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/shubham/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/shubham/.local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/shubham/.local/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 148, in s

In [46]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())